In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time
import pylas
from sklearn.cluster import DBSCAN

import src.fusion as fusion
import src.utils.las_utils as las_utils
from src.utils.labels import Labels

In [ ]:
# Data folder for the BGT fuser.
bgt_data_folder = '../datasets/bgt/'
tile_code = '2386_9702'

# Building fuser using BGT building footprint data.
bgt_point_fuser = fusion.BGTPointFuser(Labels.TREE, bgt_folder=bgt_data_folder)

In [ ]:
# Extract pole objects from tile.
bgt_points = bgt_point_fuser._filter_tile(tile_code)
trees = [(x, y) for (t, x, y) in bgt_points if t == 'boom']
poles = [(x, y) for (t, x, y) in bgt_points if t == 'lichtmast']

In [ ]:
# Load labeled and grown LAS file.
las_file = '../datasets/pointcloud/grown_' + tile_code + '.laz'
las = pylas.read(las_file)

In [ ]:
# Remove ground and building points.
mask = (las.label != Labels.GROUND) & (las.label != Labels.BUILDING)
points = {'x': las.x[mask], 'y': las.y[mask], 'z': las.z[mask]}

# Compute average ground elevation.
ground_mask = las.label == Labels.GROUND
avg_ground_height = np.mean(las.z[ground_mask])

In [ ]:
# Define a plane to search for seed points.
plane_height = 2
plane_buffer = 0.2
seed_plane_mask = (((avg_ground_height + plane_height - plane_buffer) < points['z']) 
                   & (points['z'] < (avg_ground_height + plane_height + plane_buffer)))

print(f'We have {np.count_nonzero(seed_plane_mask)} potential seed points')

In [ ]:
# Get <X, Y> of potential seed points.
points_xy = np.vstack((points['x'][seed_plane_mask], points['y'][seed_plane_mask])).T

In [ ]:
# Cluster the potential seed points.
clustering = DBSCAN(eps=0.5, min_samples=5, p=2).fit(points_xy)

# Remove noise points.
noise_mask = clustering.labels_ != -1

In [ ]:
# Optional: filter clusters by size.

# Get cluster labels and sizes.
cc_labels, counts = np.unique(clustering.labels_, return_counts=True)

# Only keep clusters with size between N_min and N_max.
N_min = 10
N_max = 5000
count_valid = np.where((counts >= N_min) & (counts <= N_max))

# Update noise mask.
noise_mask = noise_mask & [l in set(cc_labels[count_valid]) for l in clustering.labels_]

In [ ]:
# Filter points based on noise mask.
points_xy_filter = points_xy[noise_mask,:]

In [ ]:
# Visualize the results.
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as patches

bbox = las_utils.get_bbox_from_tile_code('2386_9702')

fig, ax = plt.subplots(1)
ax.scatter(points_xy_filter[:,0], points_xy_filter[:,1], c='lightgrey', marker='.')

for tree in trees:
    plt.scatter(tree[0], tree[1], c='green', marker='x')

for pole in poles:
    plt.scatter(pole[0], pole[1], c='purple', marker='x')

((x_min, y_max), (x_max, y_min)) = bbox
box = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, linestyle='--', edgecolor='grey', fill=False)
ax.add_patch(box)

ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.axis('equal')
plt.show()